# Derivations
Specifically of discretizing and linearizing continuous time dynamics as well as quadraticizing costs.

References:
1. [Jackson, Howell - iLQR Tutorial](http://roboticexplorationlab.org/papers/iLQR_Tutorial.pdf)
1. [Kavuncu et. al. - Potential iLQR](https://arxiv.org/pdf/2107.04926.pdf)
1. [Pyomo - 2.6 Model Predictive Control of a Double Integrator](https://jckantor.github.io/ND-Pyomo-Cookbook/02.06-Model-Predictive-Control-of-a-Double-Integrator.html)
1. [Discretizing an SSM](https://en.wikipedia.org/wiki/Discretization#Discretization_of_linear_state_space_models)
1. [Drake - ilqr_driving](https://colab.research.google.com/github/RussTedrake/underactuated/blob/master/exercises/trajopt/ilqr_driving/ilqr_driving.ipynb#scrollTo=WCdJhqQqHLii)
1. [Al - Augmenting an Angular State](https://github.com/anassinator/ilqr/blob/master/ilqr/examples/cartpole.py)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym
from IPython.display import display, Math

In [2]:
%matplotlib ipympl
np.set_printoptions(precision=3, suppress=True)

## Dynamical System [1]
A dynamical system can be modeled by the following non-linear, continuous dynamics:

$$\dot x = f_c(x, u)$$

Discretizing using the [following method](https://en.wikipedia.org/wiki/Discretization#Discretization_of_linear_state_space_models):

$$x_{k+1} = f(x_k, u_k)$$

Using a 1st order Taylor Series expansion, we linearize around an operating point:

$$\begin{aligned}
x_{k+1} + \delta x_{k+1} &\approx f(x_k, u_k) + 
\left.\frac{\partial f}{\partial x}\right|_{x_k, u_k}(x - x_k) + 
\left.\frac{\partial f}{\partial u}\right|_{x_k, u_k}(u - u_k)
\\
\delta x_{k+1} &= A \delta x_k + B \delta u_k
\end{aligned}
$$

In [3]:
def discretize(A, B, Δt):
    """Compute the discretization of continuous dynamics assuming zero-order hold."""
    
    k, t = sym.symbols('k t')
    Ad = sym.exp(A * Δt).simplify()
    # Ad_approx = sym.summation(1/sym.factorial(k) * (A*Δt) ** k, (k, 0, sym.oo))
    Ad_approx = sym.eye(A.shape[0], A.shape[0]) + A * Δt
    # Bd = A**-1 * (Ad - sym.eye(4,4)) * B # for non-singular A
    Bd = sym.integrate(Ad, (t, 0, Δt)) * B
    
    return Ad, Bd

In [4]:
def linearize(x_dot, x, u):
    """Compute the linearization of the dynamics with respect to x and u."""
    
    A = x_dot.jacobian(x)
    B = x_dot.jacobian(u)
    return A, B

## Unicycle Dynamics [2]
A unicycle can be modelled using the following continuous dynamics:

$$\begin{aligned}
x &= \begin{bmatrix} p_x & p_y & v & \theta \end{bmatrix} \\
u &= \begin{bmatrix} a & \omega\end{bmatrix}
\end{aligned}$$


$$\begin{aligned}
\dot p_{x} &= v\cos\theta \\
\dot p_{y} &= v\sin\theta \\
\dot \theta &= \omega \\
\dot v &= a
\end{aligned}$$

In [5]:
Δt = sym.Symbol('\Delta t')
p_x, p_y, v, theta, omega, a = sym.symbols('p_x p_y v theta omega a')

x = sym.Matrix([p_x, p_y, v, theta])
u = sym.Matrix([a, omega])
display(Math('x = ' + sym.latex(x) + r'\\u = ' + sym.latex(u)))

<IPython.core.display.Math object>

In [6]:
x_dot = sym.Matrix([
    x[2]*sym.cos(x[3]),
    x[2]*sym.sin(x[3]),
    u[0],
    u[1],
])

A, B = linearize(x_dot, x, u)
print('Continuous Time:')
display(Math('A = ' + sym.latex(A) + r'\\B = ' + sym.latex(B)))

Ad, Bd = discretize(A, B, Δt)
print('Discrete Time:')
display(Math('A_d = ' + sym.latex(Ad) + r'\\B_d = \Delta t ' + sym.latex(Bd/Δt)))

print('Others:')
Bo = sym.Matrix([[0, 0], [0, 0], [Δt, 0], [0, Δt]])
display(Math('B = \Delta t' + sym.latex(Bo/Δt)))

Continuous Time:


<IPython.core.display.Math object>

Discrete Time:


<IPython.core.display.Math object>

Others:


<IPython.core.display.Math object>

## Car Dynamics
- "car" dynamics represent a simplified version of the unicycle dynamics where the linear velocity is in the control, rather than the state.
$$\begin{aligned}
x &= \begin{bmatrix} p_x & p_y & \theta \end{bmatrix} \\
u &= \begin{bmatrix} v & \omega\end{bmatrix}
\end{aligned}$$


$$\begin{aligned}
\dot p_{x} &= v\cos\theta \\
\dot p_{y} &= v\sin\theta \\
\dot \theta &= \omega \\
\end{aligned}$$

In [7]:
delta_t = sym.Symbol('\Delta t')
p_x, p_y, v, theta, omega = sym.symbols('p_x p_y v theta omega')

x = sym.Matrix([p_x, p_y, theta])
u = sym.Matrix([v, omega])
display(Math('x = ' + sym.latex(x) + r'\\u = ' + sym.latex(u)))

<IPython.core.display.Math object>

In [8]:
x_dot = sym.Matrix([
    u[0]*sym.cos(x[2]),
    u[0]*sym.sin(x[2]),
    u[1],
])

A, B = linearize(x_dot, x, u)
print('Continuous time:')
display(Math('A = ' + sym.latex(A) + r'\\B = ' + sym.latex(B)))

Ad, Bd = discretize(A, B, Δt)
print('Discrete Time:')
display(Math('A_d = ' + sym.latex(Ad) + r'\\B_d = \Delta t ' + sym.latex(Bd/Δt)))

Continuous time:


<IPython.core.display.Math object>

Discrete Time:


<IPython.core.display.Math object>

## Bicycle Dynamics [5]
- We can add an additional component to the standard car/unicycle dynamics via steering.
- Letting $\phi$ be the steering angle, the dynamics are:
$$\begin{aligned}
x &= \begin{bmatrix} p_x & p_y & \theta & v & \phi \end{bmatrix} \\
u &= \begin{bmatrix} a & \rho\end{bmatrix}
\end{aligned}$$


$$\begin{aligned}
\dot p_{x} &= v\cos\theta \\
\dot p_{y} &= v\sin\theta \\
\dot v &= a \\
\dot \phi &= \rho \\
\end{aligned}$$

In [9]:
delta_t = sym.Symbol('\Delta t')
p_x, p_y, theta, v, phi, a, rho = sym.symbols('p_x p_y theta v phi a rho')

x = sym.Matrix([p_x, p_y, theta, v, phi])
u = sym.Matrix([a, rho])
display(Math('x = ' + sym.latex(x) + r'\\u = ' + sym.latex(u)))

<IPython.core.display.Math object>

In [10]:
x_dot = sym.Matrix([
    x[3]*sym.cos(x[2]),
    x[3]*sym.sin(x[2]),
    x[3]*sym.tan(x[4]),
    u[0],
    u[1]
])

A, B = linearize(x_dot, x, u)
print('Continuous time:')
display(Math('A = ' + sym.latex(A) + r'\\B = ' + sym.latex(B)))

Ad, Bd = discretize(A, B, Δt)
print('Discrete Time:')
display(Math('A_d = ' + sym.latex(Ad) + r'\\B_d = \Delta t ' + sym.latex(Bd/Δt)))

Continuous time:


<IPython.core.display.Math object>

Discrete Time:


<IPython.core.display.Math object>

In [11]:
Bd.simplify()
Bd / Δt

Matrix([
[\Delta t*(-\Delta t*v*sin(theta)*tan(phi)/2 + cos(theta)), -\Delta t**2*v**2*sin(theta)/(2*cos(phi)**2)],
[ \Delta t*(\Delta t*v*cos(theta)*tan(phi)/2 + sin(theta)),  \Delta t**2*v**2*cos(theta)/(2*cos(phi)**2)],
[                                        \Delta t*tan(phi),                       \Delta t*v/cos(phi)**2],
[                                                        1,                                            0],
[                                                        0,                                            1]])

## Double Integrator Dynamics [3]
- The double integrator model is a canonical second order linear system often used to demostrate control principles. 
- This can be represented in the discretized equation as:
$$x_{k+1} = A x_k + B u_k 
= \begin{bmatrix} 1 & \Delta t \\ 0 & 1\end{bmatrix} x_k + \begin{bmatrix} \frac{\Delta t^2}{2} \\ \Delta t \end{bmatrix} u_k$$

In [15]:
Δt = sym.Symbol('\Delta t')
p_x, p_y, v_x, v_y, a_x, a_y = sym.symbols('p_x p_y v_x v_y a_x a_y')

x = sym.Matrix([p_x, v_x])
u = sym.Matrix([a_x])
display(Math('x = ' + sym.latex(x) + r'\\u = ' + sym.latex(u)))

<IPython.core.display.Math object>

In [19]:
x_dot = sym.Matrix([
    v_x + a_x*Δt,
    # v_y + a_y*Δt,
    a_x,
    # a_y
])

A = x_dot.jacobian(x)
B = x_dot.jacobian(u)
discretize(A, B, Δt)

(Matrix([
 [1, \Delta t],
 [0,        1]]),
 Matrix([
 [2*\Delta t**2],
 [     \Delta t]]))

## Augmented Unicycle Dynamics [2]
A unicycle can be modelled using the following augmented continuous dynamics:

$$\begin{aligned}
x &= \begin{bmatrix} p_x & p_y & v & \sin(\theta) & \cos(\theta) \end{bmatrix} \\
u &= \begin{bmatrix} a & \omega\end{bmatrix}
\end{aligned}$$


$$\begin{aligned}
\dot p_{x} &= v\cos\theta \\
\dot p_{y} &= v\sin\theta \\
\dot \theta &= \omega \\
\dot v &= a
\end{aligned}$$

In [12]:
Δt = sym.Symbol('\Delta t')
sθ = sym.Symbol('\sin \Theta')
cθ = sym.Symbol('\cos \Theta')
p_x, p_y, v, omega, a = sym.symbols('p_x p_y v omega a')

x = sym.Matrix([p_x, p_y, v, sθ, cθ])
u = sym.Matrix([a, omega])
display(Math('x = ' + sym.latex(x) + r'\\u = ' + sym.latex(u)))

<IPython.core.display.Math object>

In [13]:
x_dot = sym.Matrix([
    x[2]*x[4],
    x[2]*x[3],
    u[0],
    u[1]*x[4], # sym.sin(u[1]),
    -u[1]*x[3] # sym.cos(u[1])
])

A, B = linearize(x_dot, x, u)
print('Continuous Time:')
display(Math('A = ' + sym.latex(A) + r'\\B = ' + sym.latex(B)))

Ad, Bd = discretize(A, B, Δt)
print('Discrete Time:')
display(Math('A_d = ' + sym.latex(Ad) + r'\\B_d = \Delta t ' + sym.latex(Bd/Δt)))

print('Others:')
Bo = sym.Matrix([[0, 0], [0, 0], [Δt, 0], [0, Δt]])
display(Math('B = \Delta t' + sym.latex(Bo/Δt)))

Continuous Time:


<IPython.core.display.Math object>

Discrete Time:


<IPython.core.display.Math object>

Others:


<IPython.core.display.Math object>

## Quadraticizing Costs

In [18]:
def quadraticize(L, x, u):
    """Quadraticize the costs with respect to the states and controls."""
    
    L_x = L.jacobian(x)
    L_u = L.jacobian(u)
    L_xx = L_x.jacobian(x)
    L_uu = L_u.jacobian(u)
    L_xu = L_x.jacobian(u)
    L_ux = L_u.jacobian(x)
    
    return L_x, L_u, L_xx, L_uu, L_xu, L_ux

In [19]:
def quadraticize_reference(x, u, x_r, Q, R, terminal=False):
    """Quadraticize reference costs analytically."""
    
    L_x = (x - x_r).T @ (Q + Q.T)
    L_u = u.T @ (R + R.T)
    L_xx = Q + Q.T
    L_uu = R + R.T
    L_xu = sym.zeros(x.shape[0], u.shape[0])
    L_ux = L_xu.T
    
    if terminal:
        γ = sym.symbols('gamma')
        L_x *= γ
        L_xx *= γ
        L_u = sym.zeros(1, u.shape[0])
        L_uu = sym.zeros(u.shape[0])
    
    return L_x, L_u, L_xx, L_uu, L_xu, L_ux

In [21]:
delta_t = sym.Symbol('\Delta t')
p_x, p_y, v, theta, omega, a = sym.symbols('p_x p_y v theta omega a')
p_xr, p_yr, v_r, theta_r, omega_r, a_r = sym.symbols('p_xr p_yr v_r theta_r omega_r a_r')
p_xo, p_yo = sym.symbols('p_xo p_yo')

x = sym.Matrix([p_x, p_y, v, theta])
u = sym.Matrix([omega, a])
x_r = sym.Matrix([p_xr, p_yr, v_r, theta_r])
x_o = sym.Matrix([p_xo, p_yo, 0, 0])

In [24]:
Q = sym.eye(x.shape[0])
R = sym.eye(u.shape[0])
# Q = sym.Matrix(np.arange(16)).reshape(4,4)
# Q = sym.diag(1, 1, 0, 0)
# Q[0,-1] = 1.0
# R = sym.diag(2.0, 1.0)
# R[0,-1] = 3.0

display(Math('Q = ' + sym.latex(Q) + '\\\R = ' + sym.latex(R)))

<IPython.core.display.Math object>

In [25]:
L = (x - x_r).T @ Q @ (x - x_r) + (u.T @ R @ u)
L_x, L_u, L_xx, L_uu, L_xu, L_ux = quadraticize(L, x, u)
L_x_a, L_u_a, L_xx_a, L_uu_a, L_xu_a, L_ux_a = quadraticize_reference(x, u, x_r, Q, R)

display(Math('L_u = ' + sym.latex(L_u) + ' = u^T (R + R^T) = ' + sym.latex(L_u_a)))
display(Math('L_x = ' + sym.latex(L_x) + ' = (x - x_r)^T (Q + Q^T) = ' + sym.latex(L_x_a)))
display(Math('L_{xx} = ' + sym.latex(L_xx) + ' = Q + Q^T = ' + sym.latex(L_xx_a)))
display(Math('L_{uu} = ' + sym.latex(L_uu) + ' = R + R^T = ' + sym.latex(L_uu_a)))
display(Math('L_{xu} = L_{ux}^T = ' + sym.latex(L_xu_a)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [19]:
dL_dxu = L_x.row_join(L_u) # Gradient
dL_dxu

Matrix([[2*p_x - 2*p_xr, 2*p_y - 2*p_yr, 0, 0, 2*omega, 2*a]])

In [20]:
dL2_dxu2 = sym.Matrix(sym.BlockMatrix([
    [L_xx, L_xu],
    [L_xu.T, L_uu]
])) # Hessian
dL2_dxu2

Matrix([
[2, 0, 0, 0, 0, 0],
[0, 2, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 2, 0],
[0, 0, 0, 0, 0, 2]])

In [21]:
γ = sym.symbols('gamma')
L_N = γ * (x - x_r).T @ Q @ (x - x_r)
L_x, L_u, L_xx, L_uu, L_xu, L_ux = quadraticize(L_N, x, u)
L_x_a, L_u_a, L_xx_a, L_uu_a, L_xu_a, L_ux_a = quadraticize_reference(x, u, x_r, Q, R, terminal=True)

display(Math('L_u = ' + sym.latex(L_u) + ' = u^T (R + R^T) = ' + sym.latex(L_u_a)))
display(Math('L_x = ' + sym.latex(L_x.T) + ' = (x - x_r)^T (Q + Q^T) = ' + sym.latex(L_x_a.T)))
display(Math('L_{xx} = ' + sym.latex(L_xx) + ' = Q + Q^T = ' + sym.latex(L_xx_a)))
display(Math('L_{uu} = ' + sym.latex(L_uu) + ' = R + R^T = ' + sym.latex(L_uu_a)))
display(Math('L_{xu} = L_{ux}^T = ' + sym.latex(L_xu_a)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Obstacle Cost
$$L = \min(0, d-d_{max})^2$$

In [22]:
from sympy.functions.elementary.piecewise import Piecewise
import matplotlib.pyplot as plt

In [23]:
d_max, d = sym.symbols('d_{max} d', positive=True)
L = Piecewise((0, d > d_max), ((d - d_max)**2, d <= d_max))

sym.plot(L.subs(d_max, 5), (d, 0, 10))
plt.title('L')
sym.plot(L.subs(d_max, 5).diff(d), (d, 0, 10))
plt.title('Lx')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Lx')

In [24]:
o_x, o_y = sym.symbols('o_x o_y')
d = sym.sqrt((x[0]-o_x)**2 + (x[1]-o_y)**2)
L = sym.Matrix([(d - d_max)**2])
L

Matrix([[(-d_{max} + sqrt((-o_x + p_x)**2 + (-o_y + p_y)**2))**2]])

In [51]:
test_args = {d_max:2, o_x:0, o_y:0, p_x:0, p_y:1}
d.subs(test_args)

1

In [48]:
L.subs(test_args)

Matrix([[1]])

In [41]:
L_x

Matrix([[2*(d_{max} - sqrt((o_x - p_x)**2 + (o_y - p_y)**2))*(o_x - p_x)/sqrt((o_x - p_x)**2 + (o_y - p_y)**2), 2*(d_{max} - sqrt((o_x - p_x)**2 + (o_y - p_y)**2))*(o_y - p_y)/sqrt((o_x - p_x)**2 + (o_y - p_y)**2), 0, 0]])

In [52]:
L_x, L_u, L_xx, L_uu, L_xu, L_ux = quadraticize(L, x, u)
L_x.simplify()
L_x.subs(test_args)

Matrix([[0, -2, 0, 0]])

In [1]:
L_xx.simplify()
# L_xx[0,1].factor()
# L_xx.subs(test_args)

NameError: name 'L_xx' is not defined